In [1]:
import pandas as pd
from PIL import Image
import torch
import os

In [2]:
data_map=pd.read_csv(r'C:\Users\ihhim\OneDrive\Desktop\project1\data\chest_xray\metadata.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_map['encoded_class'] = le.fit_transform(data_map['class'])

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms,models

In [21]:
class xraydataset(Dataset):
    def __init__(self,df_split,transform):
        self.df_split=df_split
        self.transform=transform
        
    def __len__(self):
        return len(self.df_split)  
    
    def __getitem__(self,index):  
        img_path=self.df_split.iloc[index]['image_path']
        label=self.df_split.iloc[index]['encoded_class']

        image=Image.open(img_path).convert('RGB')
        img_trans=self.transform(image)

        return img_trans,label


In [22]:
train_dataset=xraydataset(data_map[data_map['split']=='train'],transform=transforms.ToTensor())
val_dataset=xraydataset(data_map[data_map['split']=='val'],transform=transforms.ToTensor())

In [23]:
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=32,shuffle=False)

In [30]:
model=models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
model.conv1=nn.Conv2d(3,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)
model.classifier=nn.Linear(in_features=1024,out_features=2,bias=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [31]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [32]:
class_counts = data_map[data_map['split']=='train']['encoded_class'].value_counts().sort_index().values
weights = torch.tensor(1.0 / class_counts, dtype=torch.float).to(device)
loss = nn.CrossEntropyLoss(weight=weights)
optimizer=optim.Adam(model.parameters(),lr=0.0001)

In [35]:
patience=0
counter=0
best_val_loss=float('inf')

from tqdm import tqdm
num_epochs=10
for epoch in range(num_epochs):
    model.train()
    running_loss=0
    for images,labels in tqdm(train_dataloader,total=len(train_dataloader)):
        images,labels=images.to(device),labels.to(device).long()
        optimizer.zero_grad()
        preds=model(images)
        loss_val=loss(preds,labels)
        loss_val.backward()
        optimizer.step()
        running_loss+=loss_val.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader):.4f}")

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            loss_batch = loss(outputs, labels)
            val_loss += loss_batch.item()
    val_loss /= len(val_dataloader)
    print(f"Validation Loss: {val_loss:.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        # Optionally save the best model
        torch.save(model.state_dict(), r'C:\Users\ihhim\OneDrive\Desktop\project1\models\best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

100%|██████████| 243/243 [00:42<00:00,  5.75it/s]


Epoch 1/10, Loss: 0.0087
Validation Loss: 0.1216


100%|██████████| 243/243 [01:06<00:00,  3.66it/s]

Epoch 2/10, Loss: 0.0065
Validation Loss: 0.3590
Early stopping triggered!


In [36]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=le.classes_))

              precision    recall  f1-score   support

      NORMAL       1.00      0.88      0.93         8
   PNEUMONIA       0.89      1.00      0.94         8

    accuracy                           0.94        16
   macro avg       0.94      0.94      0.94        16
weighted avg       0.94      0.94      0.94        16



before the earlystopping there was an overfitting with precision,recall,f1-score and accuracy giving values equal to 1.00


In [37]:
torch.save(model.state_dict(), r'C:\Users\ihhim\OneDrive\Desktop\project1\models\desnet121.pth')